# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

R. v. Boekel  ->  R. v. Boekel  |  ['R. v. Boekel']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
E. Bañados  ->  E. Bañados  |  ['E. Bañados']
A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
M. Habouzit  ->  M. Habouzit  |  ['M. Habouzit']
Arxiv has 47 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2405.20440


extracting tarball to tmp_2405.20440...

 done.
Retrieving document from  https://arxiv.org/e-print/2405.20689
extracting tarball to tmp_2405.20689...

 done.
Retrieving document from  https://arxiv.org/e-print/2405.21054


extracting tarball to tmp_2405.21054...

 done.


Found 174 bibliographic references in tmp_2405.21054/main_final.bbl.
syntax error in line 11: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2405.21054-b31b1b.svg)](https://arxiv.org/abs/2405.21054) | **The First Billion Years, According to JWST**  |
|| A. Adamo, et al. -- incl., <mark>E. Bañados</mark>, <mark>A. d. Graaff</mark>, <mark>M. Habouzit</mark> |
|*Appeared on*| *2024-06-03*|
|*Comments*| *review article written by the attendees of the 2024 ISSI breakthrough workshop "The first billion year of the Universe", submitted. Comments welcome*|
|**Abstract**|            With stunning clarity, JWST has revealed the Universe's first billion years. The scientific community is analyzing a wealth of JWST imaging and spectroscopic data from that era, and is in the process of rewriting the astronomy textbooks. Here, 1.5 years into the JWST science mission, we provide a snapshot of the great progress made towards understanding the initial chapters of our cosmic history. We highlight discoveries and breakthroughs, topics and issues that are not yet understood, and questions that will be addressed in the coming years, as JWST continues its revolutionary observations of the Early Universe. While this compendium is written by a small number of authors, invited to ISSI Bern in March 2024 as part of the 2024 ISSI Breakthrough Workshop, we acknowledge the work of a large community that is advancing our collective understanding of the evolution of the Early Universe.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2405.20440-b31b1b.svg)](https://arxiv.org/abs/2405.20440) | **Characterization of a Longwave HgCdTe GeoSnap Detector**  |
|| R. Bowens, et al. -- incl., <mark>R. v. Boekel</mark> |
|*Appeared on*| *2024-06-03*|
|*Comments*| *16 pages, 9 figures, Submitted to SPIE Astronomical Telescopes + Instrumentation 2024*|
|**Abstract**|            New longwave HgCdTe detectors are critical to upcoming plans for ground-based infrared astronomy. These detectors, with fast-readouts and deep well-depths, will be key components of extremely large telescope instruments and therefore must be well understood prior to deployment. We analyze one such HgCdTe detector, a Teledyne Imaging Sensors GeoSnap, at the University of Michigan. We find that the properties of the GeoSnap are consistent with expectations from analysis of past devices. The GeoSnap has a well-depth of 2.75 million electrons per pixel, a read noise of 360 e-/pix, and a dark current of 330,000 e-/s/pix at 45 K. The device experiences 1/f noise which can be mitigated relative to half-well shot noise with modest frequency image differencing. The GeoSnap's quantum efficiency is calculated to be 79.7 +- 8.3 % at 10.6 microns. Although the GeoSnap's bad pixel fraction, on the order of 3%, is consistent with other GeoSnap devices, close to a third of the bad pixels in this detector are clustered in a series of 31 "leopard" spots spread across the detector plane. We report these properties and identify additional analyses that will be performed on future GeoSnap detectors.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2405.20689-b31b1b.svg)](https://arxiv.org/abs/2405.20689) | **Identifying and Fitting Eclipse Maps of Exoplanets with Cross-Validation**  |
|| M. Hammond, et al. -- incl., <mark>L. Kreidberg</mark> |
|*Appeared on*| *2024-06-03*|
|*Comments*| *Submitted to MNRAS*|
|**Abstract**|            Eclipse mapping uses the shape of the eclipse of an exoplanet to measure its two-dimensional structure. Light curves are mostly composed of longitudinal information, with the latitudinal information only contained in the brief ingress and egress of the eclipse. This imbalance can lead to a spuriously confident map, where the longitudinal structure is constrained by out-of-eclipse data and the latitudinal structure is wrongly determined by the priors on the map. We present a new method to address this issue. The method tests for the presence of an eclipse mapping signal by using k-fold cross-validation to compare the performance of a simple mapping model to the null hypothesis of a uniform disk. If a signal is found, the method fits a map with more degrees of freedom, optimising its information content. The information content is varied by penalising the model likelihood by a factor proportional to the spatial entropy of the map, optimised by cross-validation. We demonstrate this method for simulated datasets then apply it to three observational datasets. The method identifies an eclipse mapping signal for JWST MIRI/LRS observations of WASP-43b but does not identify a signal for JWST NIRISS/SOSS observations of WASP-18b or Spitzer Space Telescope observations of HD 189733b. It is possible to fit eclipse maps to these datasets, but we suggest that these maps are overfitting the eclipse shape. We fit a new map with more spatial freedom to the WASP-43b dataset and show a flatter east-west structure than previously derived.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2405.21054.md
    + _build/html/tmp_2405.21054/./RedshiftCompilation-v10.png
    + _build/html/tmp_2405.21054/./JWST-Paper-3dBubbles-v5.png
    + _build/html/tmp_2405.21054/./BeginningOfUniverse-Draft4.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\grizli}{\textsc{grizli}}$
$\newcommand{\JWST}{\textit{JWST}}$
$\newcommand{\red}[1]{{\color{red} #1}}$
$\newcommand{\todo}[1]{\red{#1}}$
$\newcommand{\mh}[1]{\textcolor{teal}{[MH: #1]}}$
$\newcommand{\msun}{M_{\odot} }$
$\newcommand{\Zsun}{\ensuremath{Z_{\odot}}}$
$\newcommand{\zphot}{\ensuremath{z_{\rm phot}}}$
$\newcommand{\fesc}{\ensuremath{f_{esc}}}$
$\newcommand{\Ha}{H\alpha}$
$\newcommand{\Hb}{H\beta}$
$\newcommand{\Hd}{H\delta}$
$\newcommand{\Hg}{H\gamma}$
$\newcommand{\HI}{H {\sc i}}$
$\newcommand{\HeII}{He {\sc ii}}$
$\newcommand{\HeIIw}{\HeII \lambda1640}$
$\newcommand{\CIV}{C {\sc iv}}$
$\newcommand{\CIVw}{\CIV \lambda1548,1550}$
$\newcommand{\OIIIuv}{O {\sc iii}]}$
$\newcommand{\OIIIuvw}{\OIIIuv \lambda1661,1666}$
$\newcommand{\NIIuv}{N {\sc ii}]}$
$\newcommand{\NIIuvw}{\NIIuv \lambda1750}$
$\newcommand{\CIII}{C {\sc iii}]}$
$\newcommand{\CIIIw}{\CIII \lambda1908}$
$\newcommand{\OII}{[O {\sc ii}]}$
$\newcommand{\OIIw}{\OII \lambda\lambda3726,28}$
$\newcommand{\NeIII}{[Ne {\sc iii}]}$
$\newcommand{\NeIIIw}{\NeIII \lambda3869}$
$\newcommand{\logOH}{12+log[O/H]}$</div>



<div id="title">

# The First Billion Years, According to JWST

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2405.21054-b31b1b.svg)](https://arxiv.org/abs/2405.21054)<mark>Appeared on: 2024-06-03</mark> -  _review article written by the attendees of the 2024 ISSI breakthrough workshop "The first billion year of the Universe", submitted. Comments welcome_

</div>
<div id="authors">

A. Adamo, et al. -- incl., <mark>E. Bañados</mark>, <mark>A. d. Graaff</mark>, <mark>M. Habouzit</mark>

</div>
<div id="abstract">

**Abstract:** With stunning clarity, $\JWST$ has revealed theUniverse's first billion years. The scientific community is analyzing a wealth of $\JWST$ imaging and spectroscopic data from that era, and is in the process of rewriting the astronomy textbooks. Here, 1.5 years into the $\JWST$ science mission, we provide a snapshot of the great progress made towards understanding the initial chapters of our cosmic history.We highlight discoveries and breakthroughs, topics and issues that are not yet understood, and questions that will be addressed in the coming years, as $\JWST$ continues its revolutionary observations of the Early Universe. While this compendium is written by a small number of authors, invited to ISSI Bern in March 2024 as part of the 2024 ISSI Breakthrough Workshop, we acknowledge the work of a large community that is advancing our collective understanding of the evolution of the Early Universe.

</div>

<div id="div_fig1">

<img src="tmp_2405.21054/./RedshiftCompilation-v10.png" alt="Fig2" width="100%"/>

**Figure 2. -** **a.** The distribution of absolute magnitudes and redshifts of spectroscopically-confirmed galaxies from pre-JWST candidates (blue dots) and from public JWST data sets (orange squares), showing the power of JWST to detect galaxies beyond redshift 6. The latter include compilations \citep{RobertsBorsani2024} and single targets \citep{castellano2024,carniani2024} observed with NIRSpec MSA observations, as well as NIRCam grism (FRESCO and EIGER; \citep{Oesch2023} and \citep{Daichi2023}, respectively).
    **b.** The cosmic SFR density over the first billion years (adapted from Figure 17 of \citep{Harikane2024}, as seen from HST/WFC3 samples (dark circles), compared to JWST/NIRCam estimates (light squares). A model of constant star formation efficiency is plotted in grey, for comparison. The model and all literature points are derived from \citep{Harikane2024}(and references therein), where the latter are integrated down to $M_{\rm UV}=-18$ mag. (*fig:MUV-z*)

</div>
<div id="div_fig2">

<img src="tmp_2405.21054/./JWST-Paper-3dBubbles-v5.png" alt="Fig6" width="100%"/>

**Figure 6. -** Spectroscopically-confirmed galaxies in the CEERS EGS field at $z = 7.1–7.8$, reproduced from \citep{Chen2024}. The presence of numerous Lyman-alpha emitting galaxies (red stars), including several with high equivalent widths ($>200$ Å) and Lyman-alpha escape fractions ($\gtrsim50\%$), in this field provides strong evidence for candidate ionized bubbles along the line of sight (shaded cyan regions -- for illustration purposes only). These early observations, primarily of $M_\mathrm{UV} < -19$ HST-selected sources, highlight the potential of JWST to create tomographic maps of ionized regions to learn about the reionization process on local scales. (*fig:bubble*)

</div>
<div id="div_fig3">

<img src="tmp_2405.21054/./BeginningOfUniverse-Draft4.png" alt="Fig1" width="100%"/>

**Figure 1. -** The cosmic timeline, from the origin of the Universe in the Big Bang, 13.8 billion years ago, till the present day. In the current standard picture,  the Universe underwent a period of accelerated expansion called "inflation" that expanded the Universe by about 60 orders of magnitude. The Universe then kept cooling and expanding until the next major epoch of "recombination" about $4\times 10^5$ yr later when the first hydrogen atoms formed. This was followed by the "Dark ages" of the Universe that lasted for a few hundred million years. The emergence of the earliest galaxies, a few hundred million years after the Big Bang, marked the start of the era of "cosmic dawn". The first galaxies also produced the first photons capable of ionizing the neutral hydrogen atoms permeating space, starting the Epoch of Reionization (EoR), the last major phase transition in the Universe. In the initial stages of reionization, isolated galaxies (light yellow dots) produced ionized regions (gray patches) that grew and merged  until the Universe was fully reionized. Image Credit: DELPHI project (ERC 717001). (*fig:universe_timeline*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2405.21054"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

82  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

5  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
